### Reference Dates

Num Training Weeks = 4  
Num Test/Val Days = 15

- Train: 5/30/2017 (Tues), 6/6/2017,6/13/2017,6/20/2017  
- Val: 7/11/2017 (Tues) - 7/25/2017 (Wed)  
- Test: 8/1/2017 (Tues)  - 8/15/2017 (Wed)  

In [6]:
!pip install tqdm
!pip install  pandas
!pip install sklearn
!pip install keras

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
from tqdm import tqdm

In [59]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras import callbacks
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import gc

In [9]:
df_train = pd.read_csv(
    '../input/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [10]:
items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "../input/stores.csv",
).set_index("store_nbr")

In [11]:
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [12]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [13]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train
gc.collect()

22

In [14]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [15]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train, df_test
gc.collect()

0

In [16]:
promo_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   

date                2017-01-09  2017-01-10  ...  2017-08-22  2017-08-23  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False  
          105574         False       False       False       False  

[5 rows x 243 columns]

In [17]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [18]:
df_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995       0.000000    0.000000  ...    1.098612    1.098612   
          99197       0.000000    0.693147  ...    0.000000    1.098612   
          103520      0.693147    0.693147  ...    0.000000    0.000000   
          103665      0.000000    2.079442  ...    0.693147    1.098612   
          105574      1.386294    2.302585  ...    0.000000    1.791759   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.693147    0.000000   
          99197       0.000000    1.098612    0.000000    0.000000   
          103520      1.386294    0.000000    1.386294    0.693147   
          103665      0.000000    2.079442    2.302585    1.098612   
          105574      2.079442    1.945910    2.397895    1.791759   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995       0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.000000    0.000000  
          103665      0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [19]:
items['class'] = items['class'].astype('category')
items = pd.get_dummies(items)

In [20]:
items.head()

,perishable,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,family_CLEANING,family_DAIRY,...,class_6920,class_6922,class_6924,class_6936,class_6954,class_6960,class_7002,class_7016,class_7034,class_7780
item_nbr,,,,,,,,,,,,,,,,,,,,,
96995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103501,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
103520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103665,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
items = items.reindex(df_2017.index.get_level_values(1))

In [22]:
items.shape

(167515, 371)

In [23]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [24]:
stores['cluster'] = stores.cluster.astype('category')
stores = pd.get_dummies(stores)
stores = stores.reindex(df_2017.index.get_level_values(0))

In [25]:
stores.head()

,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_El Carmen,city_Esmeraldas,city_Guaranda,city_Guayaquil,city_Ibarra,...,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17
store_nbr,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [26]:
stores.shape

(167515, 60)

In [27]:
#sum of sales for each item across all stores for each day
#df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()

In [28]:
#sum of promotion for each item across all stores for each day
#promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

In [29]:
# sum of sales in each class (multiple items) for each store for each day 
# df_2017_store_class = df_2017.reset_index()
# df_2017_store_class['class'] = items['class'].values
# df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
# df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

In [30]:
#df_2017_store_class_index.head()

In [31]:
#df_2017_store_class.shape

In [32]:
#df_2017_store_class.head()

In [33]:
# sum of promotions in each class (multiple items) for each store for each day 

# df_2017_promo_store_class = promo_2017.reset_index()
# df_2017_promo_store_class['class'] = items['class'].values
# df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
# df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

In [34]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [35]:
def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "promo_7_2017": get_timespan(promo_df, t2017, 7, 7).sum(axis=1).values,
        "promo_14_2017": get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,
        "promo_30_2017": get_timespan(promo_df, t2017, 30, 30).sum(axis=1).values,
        "promo_3_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 3).sum(axis=1).values,
        "promo_7_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 7).sum(axis=1).values,
        "promo_14_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 14).sum(axis=1).values,
    }

# Removed due to the presence of nan values 
#     for i in [3, 7, 14, 30]:
#         tmp1 = get_timespan(df, t2017, i, i)
#         tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1

#         X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
#         X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values
                
        
    for i in [3, 7, 14, 30]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

    for i in [7, 14, 30]:
        tmp = get_timespan(df, t2017, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

        tmp = get_timespan(promo_df, t2017, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    tmp = get_timespan(promo_df, t2017 + timedelta(days=15), 14, 14)
    X['has_promo_days_in_after_14_days'] = (tmp > 0).sum(axis=1).values
    X['last_has_promo_day_in_after_14_days'] = i - ((tmp > 0) * np.arange(14)).max(axis=1).values
    X['first_has_promo_day_in_after_14_days'] = ((tmp > 0) * np.arange(14, 0, -1)).max(axis=1).values

    for i in range(1, 15):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values        
        
    for i in range(-14, 15):
        X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = df[
            pd.date_range(t2017, periods=15)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X

In [36]:
from tqdm import tqdm

In [37]:
print("Preparing dataset...")
t2017 = date(2017, 5, 30)
num_days = 4
X_l, y_l = [], []
for i in tqdm(range(num_days)):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)

    X_tmp = pd.concat([X_tmp, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

del X_l, y_l
gc.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

Preparing dataset...


100%|██████████| 4/4 [00:09<00:00,  2.28s/it]


0

In [38]:
X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 11))

# X_val2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 7, 26), is_train=False, name_prefix='item')
# X_val2.index = df_2017_item.index
# X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

# X_val3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 7, 26), is_train=False, name_prefix='store_class')
# X_val3.index = df_2017_store_class.index
# X_val3 = X_val3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_val = pd.concat([X_val, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)

X_test, y_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 1))

# X_test2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 8, 16), is_train=False, name_prefix='item')
# X_test2.index = df_2017_item.index
# X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

# X_test3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 8, 16), is_train=False, name_prefix='store_class')
# X_test3.index = df_2017_store_class.index
# X_test3 = X_test3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_test = pd.concat([X_test, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)

# del X_test2, X_val2, df_2017_item, promo_2017_item, df_2017_store_class, df_2017_promo_store_class, df_2017_store_class_index
# gc.collect()

In [39]:
for col in X_train.columns:
    if X_train[col].nunique() == 1:
        print(col)

class_1005
class_1328
class_1334


In [40]:
#there are columns with unique values but will leave it here for simplicity

In [41]:
X_train.shape, y_train.shape

((670060, 539), (670060, 15))

In [42]:
X_val.shape, y_val.shape

((167515, 539), (167515, 15))

In [43]:
X_test.shape, y_test.shape

((167515, 539), (167515, 15))

In [44]:
del df_2017, promo_2017
gc.collect()

0

In [45]:
scaler = StandardScaler()
scaler.fit(pd.concat([X_train, X_val, X_test]))
X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
X_test[:] = scaler.transform(X_test)

In [46]:
X_train = X_train.values
X_test = X_test.values
X_val = X_val.values

In [47]:
X_train.shape, y_train.shape

((670060, 539), (670060, 15))

In [48]:
X_val.shape, y_val.shape

((167515, 539), (167515, 15))

In [49]:
X_test.shape, y_test.shape

((167515, 539), (167515, 15))

## MLP Model 3

256 x 128 x 64 x 32 x 16 x 1

In [60]:
def build_model():

    model = Sequential()

    model.add(Dense(256, input_dim=X_train.shape[1]))
    model.add(PReLU())
    model.add(BatchNormalization())
#     model.add(Dropout(.1))

    model.add(Dense(128, input_dim=X_train.shape[1]))
    model.add(PReLU())
    model.add(BatchNormalization())
#     model.add(Dropout(.05))
#     model.add(Dropout(0.1))
    
    model.add(Dense(64))
    model.add(PReLU())
    model.add(BatchNormalization())
#     model.add(Dropout(.05))

    model.add(Dense(32))
    model.add(PReLU())
    model.add(BatchNormalization())
#     model.add(Dropout(.05))

    model.add(Dense(16))
    model.add(PReLU())
    model.add(BatchNormalization())
#     model.add(Dropout(.05))

    model.add(Dense(1))
    return model

In [62]:
N_EPOCHS = 2000

val_pred = []
test_pred = []
# wtpath = 'weights.hdf5'  # To save best epoch. But need Keras bug to be fixed first.
sample_weights=np.array( pd.concat([items["perishable"]] * num_days) * 0.25 + 1 )
for i in range(15):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    y = y_train[:, i]
    y_mean = y.mean()
    xv = X_val
    yv = y_val[:, i]
    model = build_model()

    opt = optimizers.Adam(lr=0.001)
#     opt = optimizers.RMSprop(learning_rate=0.001, rho=0.9)
    model.compile(loss='mse', optimizer=opt, metrics=['mse'])

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
        ]
    
    #smaller batch size runs faster
    #batch_size = 65536
    batch_size = 8192

    model.fit(X_train, y - y_mean, batch_size = batch_size, epochs = N_EPOCHS, verbose=2,
               sample_weight=sample_weights, validation_data=(xv,yv-y_mean), callbacks=callbacks )
    val_pred.append(model.predict(X_val)+y_mean)
    test_pred.append(model.predict(X_test)+y_mean)



Step 1
Train on 670060 samples, validate on 167515 samples
Epoch 1/2000
 - 20s - loss: 0.4925 - mse: 0.4638 - val_loss: 0.3851 - val_mse: 0.3851
Epoch 2/2000
 - 18s - loss: 0.3548 - mse: 0.3347 - val_loss: 0.3368 - val_mse: 0.3368
Epoch 3/2000
 - 18s - loss: 0.3387 - mse: 0.3199 - val_loss: 0.3299 - val_mse: 0.3299
Epoch 4/2000
 - 18s - loss: 0.3302 - mse: 0.3121 - val_loss: 0.3228 - val_mse: 0.3228
Epoch 5/2000
 - 18s - loss: 0.3247 - mse: 0.3071 - val_loss: 0.3210 - val_mse: 0.3210
Epoch 6/2000
 - 18s - loss: 0.3207 - mse: 0.3033 - val_loss: 0.3186 - val_mse: 0.3186
Epoch 7/2000
 - 18s - loss: 0.3175 - mse: 0.3004 - val_loss: 0.3190 - val_mse: 0.3190
Epoch 8/2000
 - 18s - loss: 0.3145 - mse: 0.2977 - val_loss: 0.3191 - val_mse: 0.3191
Epoch 9/2000
 - 18s - loss: 0.3122 - mse: 0.2955 - val_loss: 0.3201 - val_mse: 0.3201
Epoch 10/2000
 - 18s - loss: 0.3099 - mse: 0.2934 - val_loss: 0.3210 - val_mse: 0.3210
Epoch 11/2000
 - 18s - loss: 0.3075 - mse: 0.2912 - val_loss: 0.3210 - val_mse: 

In [63]:
print('finished')

finished


In [64]:
weight = items["perishable"] * 0.25 + 1
val_err = (y_val - np.array(val_pred).squeeze(axis=2).transpose())**2
val_err = val_err.sum(axis=1) * weight
#change to 15 days
val_err = np.sqrt(val_err.sum() / weight.sum() / 15)
print('nwrmsle = {}'.format(val_err))

nwrmsle = 0.6099957913412681


In [65]:
test_err = (y_test - np.array(test_pred).squeeze(axis=2).transpose())**2
test_err = test_err.sum(axis=1) * weight
#change to 15 days
test_err = np.sqrt(test_err.sum() / weight.sum() / 15)
print('nwrmsle = {}'.format(test_err))

nwrmsle = 0.6223143411577241


In [87]:
df_2017_index = pd.read_csv('../derived_datasets/df_2017_index.csv',header=None)

In [88]:
df_2017_index = df_2017_index.set_index([0,1]).index

In [89]:
df_2017_index.rename(['store_nbr','item_nbr'],inplace=True)

In [90]:
df_pred_test = pd.DataFrame(np.array(test_pred).squeeze(axis=2).transpose(), 
                            index = df_2017_index, columns = pd.date_range('2017-08-01',periods=15))

In [91]:
df_pred_test.to_csv('../derived_datasets/nn_test_pred_model_3.csv')